In [1]:
import pandas as pd
import os
os.chdir('../')

In [2]:
df = pd.read_csv('output/input.csv')

Number of emergency covid hospital admissions and those that can be identified in A&E attendances

In [4]:
num_emergency_cov_admissions = df['emergency_covid_hospital_admission_date'].count()
ae_hosp_before = df["ae_before_with_hospital_discharge_date"].count()
ae_hosp_cov_before = df["ae_before_with_hospital_discharge_covid"].sum()
ae_hosp_resp_before = df["ae_before_with_hospital_discharge_resp"].sum()
ae_hosp_prim_care_probable_before = df["ae_before_with_hospital_discharge_prim_care_probable"].sum()
ae_hosp_prim_care_suspected_before = df["ae_before_with_hospital_discharge_prim_care_suspected"].sum()
ae_hosp_pos_test_before = df["ae_before_with_hospital_discharge_positive_covid_test"].sum()

print(f'Emergency covid hospital admissions: {num_emergency_cov_admissions}')
print(f'With A&E attendance and hospital discharge: {ae_hosp_before}')
print(f'With A&E attendance, hospital discharge and covid code: {ae_hosp_cov_before}')
print(f'With A&E attendance, hospital discharge and resp or covid code: {ae_hosp_resp_before}')
print(f'With A&E attendance, hospital discharge and pos test before: {ae_hosp_pos_test_before}')
print(f'With A&E attendance, hospital discharge and suspected covid in primary care: {ae_hosp_prim_care_suspected_before}')
print(f'With A&E attendance, hospital discharge and covid in primary care: {ae_hosp_prim_care_probable_before}')


Emergency covid hospital admissions: 300
With A&E attendance and hospital discharge: 500
With A&E attendance, hospital discharge and covid code: 500
With A&E attendance, hospital discharge and resp or covid code: 500
With A&E attendance, hospital discharge and pos test before: 500
With A&E attendance, hospital discharge and suspected covid in primary care: 500
With A&E attendance, hospital discharge and covid in primary care: 500


False positives

In [12]:
fp_ae_hosp_discharge = sum(df["emergency_covid_hospital_admission_after_ae_hosp_discharge"]==0)
fp_ae_hosp_discharge_cov = sum(df["emergency_covid_hospital_admission_after_ae_hosp_discharge_cov_code"]==0)
fp_ae_hosp_discharge_resp = sum(df["emergency_covid_hospital_admission_after_ae_hosp_discharge_resp_code"]==0)
fp_ae_hosp_discharge_test = sum(df["emergency_covid_hospital_admission_after_ae_hosp_discharge_pos_test"]==0)
fp_ae_hosp_discharge_suspected = sum(df["emergency_covid_hospital_admission_after_ae_hosp_discharge_cov_suspected_prim_care"]==0)
fp_ae_hosp_discharge_probable = sum(df["emergency_covid_hospital_admission_after_ae_hosp_discharge_cov_prim_care"]==0)

print(f'With A&E attendance and hospital discharge: {fp_ae_hosp_discharge}')
print(f'With A&E attendance, hospital discharge and covid code: {fp_ae_hosp_discharge_cov}')
print(f'With A&E attendance, hospital discharge and resp or covid code: {fp_ae_hosp_discharge_resp}')
print(f'With A&E attendance, hospital discharge and pos test before: {fp_ae_hosp_discharge_test}')
print(f'With A&E attendance, hospital discharge and suspected covid in primary care: {fp_ae_hosp_discharge_suspected}')
print(f'With A&E attendance, hospital discharge and covid in primary care: {fp_ae_hosp_discharge_probable}')


With A&E attendance and hospital discharge: 700
With A&E attendance, hospital discharge and covid code: 500
With A&E attendance, hospital discharge and resp or covid code: 500
With A&E attendance, hospital discharge and pos test before: 500
With A&E attendance, hospital discharge and suspected covid in primary care: 500
With A&E attendance, hospital discharge and covid in primary care: 500
